# PyKale Tutorial: Domain Adaptation on Digits with Lightning

| [Open in Colab](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/digits_dann/tutorial.ipynb) (click `Runtime` → `Run all (Ctrl+F9)` | [Launch Binder](https://mybinder.org/v2/gh/pykale/pykale/HEAD?filepath=examples%2Fdigits_dann%2Ftutorial.ipynb) (click `Run` → `Run All Cells`) |

If using [Google Colab](https://colab.research.google.com), a free GPU can be enabled to save time via setting `Runtime` → `Change runtime type` → `Hardware accelerator: GPU`

## Introduction

[Domain Adaptation](https://en.wikipedia.org/wiki/Domain_adaptation) takes a model trained and evaluated on one set of data (the source) and adapts it to another (the target). In this tutorial, a model is trained on one digits dataset (source) and adapted to another (target). This tutorial is constructed based on the `digits_dann` example `main.py`, which is in turn refactored from the [ADA: (Yet) Another Domain Adaptation library](https://github.com/criteo-research/pytorch-ada). It has been put together to run interactively on online hosting platforms including [Google Colab](https://colab.research.google.com) or [myBinder](https://mybinder.org), but can also be downloaded and run locally. Follow the [PyKale installation instructions](https://pykale.readthedocs.io/en/latest/installation.html) for this.

## Setup

The first few blocks of code are necessary to set up the notebook execution environment and import the required modules, including PyKale.

This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip uninstall --yes imgaug && pip uninstall --yes albumentations && pip install git+https://github.com/aleju/imgaug.git
    !pip install numpy>=2.0.0
    !git clone https://github.com/pykale/pykale.git
    %cd pykale
    !pip install .[image,example] 
    %cd examples/digits_dann
    !pip install tensorboard
else:
    print('Not running on CoLab')

This imports required modules.

In [ ]:
from config import get_cfg_defaults
import pytorch_lightning as pl
import torch

from model import get_model
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger

from kale.loaddata.image_access import DigitDataset, ImageAccess
from kale.loaddata.multi_domain import BinaryDomainDatasets, MultiDomainAccess, MultiDomainDataset
from kale.utils.seed import set_seed

/home/shuo/anaconda3/envs/pykale/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Configuration

In this tutorial we modify the [default configuration for domain adaptation problems](https://github.com/pykale/pykale/blob/main/examples/digits_dann/config.py) with a customized [`.yaml` file for the specific application in this tutorial](https://github.com/pykale/pykale/blob/main/examples/digits_dann/configs/TUTORIAL.yaml). The configuration is summarized below the following cell.

In [ ]:
cfg_path = "./configs/tutorial.yaml" # Path to `.yaml` config file

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
print(cfg)

COMET:
  API_KEY: 
  ENABLE: False
  EXPERIMENT_NAME: DigitDANN
  PROJECT_NAME: Digit DANN
DAN:
  METHOD: DANN
  RANDOM_DIM: 1024
  USERANDOM: False
DATASET:
  DIMENSION: 784
  NAME: digits
  NUM_CLASSES: 10
  NUM_REPEAT: 1
  ROOT: ./data
  SIZE_TYPE: source
  SOURCE: mnist
  TARGET: usps
  VALID_SPLIT_RATIO: 0.5
  WEIGHT_TYPE: natural
OUTPUT:
  OUT_DIR: outputs
  PB_FRESH: 0
  VERBOSE: False
SOLVER:
  AD_LAMBDA: True
  AD_LR: True
  BASE_LR: 0.001
  INIT_LAMBDA: 1.0
  LOG_EVERY_N_STEPS: 10
  MAX_EPOCHS: 3
  MIN_EPOCHS: 0
  MOMENTUM: 0.9
  NESTEROV: True
  NUM_WORKERS: 0
  SEED: 2025
  TEST_BATCH_SIZE: 200
  TRAIN_BATCH_SIZE: 150
  TYPE: SGD
  WEIGHT_DECAY: 0.0005


## Check if a GPU is available

If a CUDA GPU is available, this should be used to accelerate the training process. The code below checks and reports on this.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using: " + device)
devices = 1 if device == "cuda" else 0

Using: cpu


## Select Datasets

Source and target datasets are specified using `DigitDataset.get_source_target` from values in the configuration (`cfg`) above. In this tutorial, we specify a subset of classes (1, 3 and 8) to make training and testing quicker.

In [ ]:
# source, target, num_channels = DigitDataset.get_source_target(
#     DigitDataset(cfg.DATASET.SOURCE.upper()), DigitDataset(cfg.DATASET.TARGET.upper()), cfg.DATASET.ROOT
# )
#
# class_subset = [1, 3, 8]
#
# dataset = BinaryDomainDatasets(
#     source,
#     target,
#     config_weight_type=cfg.DATASET.WEIGHT_TYPE,
#     config_size_type=cfg.DATASET.SIZE_TYPE,
#     valid_split_ratio=cfg.DATASET.VALID_SPLIT_RATIO,
#     class_ids=class_subset,
# )
data_src = DigitDataset(cfg.DATASET.SOURCE.upper())
data_tgt = DigitDataset(cfg.DATASET.TARGET.upper())
num_channels = max(DigitDataset.get_channel_numbers(data_src), DigitDataset.get_channel_numbers(data_tgt))
data_access = MultiDomainAccess(
    {
        cfg.DATASET.SOURCE.upper(): DigitDataset.get_access(data_src, cfg.DATASET.ROOT)[0],
        cfg.DATASET.TARGET.upper(): DigitDataset.get_access(data_tgt, cfg.DATASET.ROOT)[0],
    },
    cfg.DATASET.NUM_CLASSES,
    return_domain_label=True,
)
# data_access = ImageAccess.get_multi_domain_images(
#     "DIGITS",
#     cfg.DATASET.ROOT,
#     sub_domain_set=[cfg.DATASET.SOURCE.upper(), cfg.DATASET.TARGET.upper()],
#     return_domain_label=True,
# )
dataset = MultiDomainDataset(data_access)

In [ ]:
cfg.DATASET.SOURCE.upper()

'MNIST'

## Set Seed

Some algorithms used in model training require generation of pseudo-random numbers. Setting the seed from which these are generated ensures reproducibility.

In [ ]:
seed = cfg.SOLVER.SEED
# seed_everything in pytorch_lightning did not set torch.backends.cudnn
set_seed(seed)

## Setup Model

Here, we use the previously defined configuration and dataset to set up the model we will subsequently train.

In [ ]:
%time model, train_params = get_model(cfg, dataset, num_channels)

CPU times: user 6.1 s, sys: 304 ms, total: 6.4 s
Wall time: 6.21 s


Output reports on data file use.

## Setup Logger

A Tensorboard logger is used to store output generated during model training. This information can be used to assess the effectiveness of the training and to identify problems. The output model is stored at `cfg.OUTPUT.OUT_DIR`.

In [ ]:
tb_logger = TensorBoardLogger(cfg.OUTPUT.OUT_DIR, name="seed{}".format(seed))

## Setup Checkpoint

A `ModelCheckpoint` is used to save the model and some quantitative measure(s) periodically.

In [ ]:
checkpoint_callback = ModelCheckpoint(filename="{epoch}-{step}-{valid_loss:.4f}", monitor="valid_loss", mode="min",)

A `TQDMProgressBar` is used to set the progress bar. `PB_FRESH` determines at which rate (in number of batches) the progress bars get updated. Set it to ``0`` to disable the display.

In [ ]:
progress_bar = TQDMProgressBar(cfg.OUTPUT.PB_FRESH)

## Setup Trainer

A trainer object is used to determine and store model parameters. Here, one is configured with information on how a model should be trained, and what hardware will be used.

In [ ]:
trainer = pl.Trainer(
    min_epochs=cfg.SOLVER.MIN_EPOCHS,
    max_epochs=cfg.SOLVER.MAX_EPOCHS,
    callbacks=[checkpoint_callback, progress_bar],
    logger=tb_logger,
    accelerator="gpu" if devices != 0 else "cpu")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Output reports on available GPU and TPU resources.

## Train Model

Optimize model parameters using the trainer.

In [ ]:
%time trainer.fit(model)

Missing logger folder: outputs/seed2025

  | Name              | Type                | Params | Mode 
------------------------------------------------------------------
0 | feat              | SmallCNNFeature     | 309 K  | train
1 | classifier        | ClassNetSmallImage  | 24.4 K | train
2 | domain_classifier | DomainNetSmallImage | 13.3 K | train
------------------------------------------------------------------
347 K     Trainable params
0         Non-trainable params
347 K     Total params
1.389     Total estimated model params size (MB)
/home/shuo/anaconda3/envs/pykale/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/shuo/anaconda3/envs/pykale/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'tr

CPU times: user 15min 2s, sys: 0 ns, total: 15min 2s
Wall time: 1min 15s


## Test Optimized Model

Check performance of model optimized with training data against test data which was not used in training.

In [ ]:
# test scores
%time trainer.test()

/home/shuo/anaconda3/envs/pykale/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at outputs/seed2025/version_0/checkpoints/epoch=2-step=1212-valid_loss=1.5365.ckpt
Loaded model weights from the checkpoint at outputs/seed2025/version_0/checkpoints/epoch=2-step=1212-valid_loss=1.5365.ckpt
/home/shuo/anaconda3/envs/pykale/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_domain_acc        0.49561181434599155
  test_domain_div_loss       1.392730712890625
        test_loss            1.525232195854187
     test_source_acc        0.9740084388185654
 test_source_domain_acc     0.3648945147679325
     test_target_acc        0.8388185654008439
 test_target_domain_acc     0.6263291139240507
     test_task_loss         0.13262920081615448
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
CPU times: user 21 s, sys: 2.88 ms, total: 21 s
Wall time: 1.76 s


[{'test_source_acc': 0.9740084388185654,
  'test_target_acc': 0.8388185654008439,
  'test_domain_acc': 0.49561181434599155,
  'test_source_domain_acc': 0.3648945147679325,
  'test_target_domain_acc': 0.6263291139240507,
  'test_loss': 1.525232195854187,
  'test_task_loss': 0.13262920081615448,
  'test_domain_div_loss': 1.392730712890625}]

Outputs are defined as:

* 'test_domain_acc': Accuracy on classifying the domain (source or target) from which data came.
* 'test_source_acc': Accuracy on test data drawn from the source dataset.
* 'test_target_acc': Accuracy on test data drawn from the target dataset.
* 'test_loss': Loss function value on the test data.